In [ ]:
import numpy as np
import pandas as pd
import os
import re
import string
import random

In [ ]:
# opening the file in read mode
my_file = open("train-source.txt", "r", encoding='UTF-8')

# reading the file
data = my_file.read()

train_source_list = data.replace('\n'," ").split('<s>')

my_file.close()

#Remove End Sentence Tags
for item in range(len(train_source_list)):
  train_source_list[item] = train_source_list[item].replace('</s>', '')


In [ ]:
train_source_list

In [ ]:
# opening the file in read mode
my_file = open("train-target.txt", "r", encoding='UTF-8')

# reading the file
data = my_file.read()

train_target_list = data.replace('\n'," ").split('<s>')

my_file.close()

#Remove End Sentence Tags
for item in range(len(train_target_list)):
  train_target_list[item] = train_target_list[item].replace('</s>', '')

In [ ]:
print("length of training sets Source: ", len(train_source_list), " Target :", len(train_target_list))

length of training sets Source:  45172  Target : 45172


In [ ]:
def make_markov_model(data, n_gram):
    markov_model = {}
    for i in range(len(data)-n_gram-1):
        curr_state, next_state = "", ""
        for j in range(n_gram):
            curr_state += data[i+j] + ""
            next_state += data[i+j]+ data[j+n_gram] + ""
        curr_state = curr_state[:-1]
        next_state = next_state[:-1]
        if curr_state not in markov_model:
            markov_model[curr_state] = {}
            markov_model[curr_state][next_state] = 1
        else:
            if next_state in markov_model[curr_state]:
                markov_model[curr_state][next_state] += 1
            else:
                markov_model[curr_state][next_state] = 1
    
    # calculating transition probabilities
    for curr_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[curr_state][state] = count/total
        
    return markov_model

In [ ]:
train_source_bigram_model = make_markov_model(train_source_list,n_gram=3)
print("number of states in bigram model for Train Source = ", len(train_source_bigram_model.keys()))

train_target_bigram_model = make_markov_model(train_target_list,n_gram=3)
print("number of states in bigram model for Train Target = ", len(train_target_bigram_model.keys()))

number of states in bigram model for Train Source =  45164
number of states in bigram model for Train Target =  45160


In [ ]:
#Anything goes ref: https://machinelearningmastery.com/calculate-bleu-score-for-text-python/
from nltk.translate.bleu_score import sentence_bleu,corpus_bleu
reference = train_source_list
candidate = train_target_list[1]

In [ ]:
print('Cumulative 1-gram: %f' % sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
print('Cumulative 2-gram: %f' % sentence_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0)))
print('Cumulative 3-gram: %f' % sentence_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0)))
print('Cumulative 4-gram: %f' % sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25)))

Cumulative 1-gram: 1.000000
Cumulative 2-gram: 1.000000
Cumulative 3-gram: 1.000000
Cumulative 4-gram: 1.000000


In [ ]:
reference = train_source_list
candidate = train_target_list

print('Cumulative 1-gram: %f' % corpus_bleu(reference, candidate, weights=(1, 0, 0, 0)))
print('Cumulative 2-gram: %f' % corpus_bleu(reference, candidate, weights=(0.5, 0.5, 0, 0)))
print('Cumulative 3-gram: %f' % corpus_bleu(reference, candidate, weights=(0.33, 0.33, 0.33, 0)))
print('Cumulative 4-gram: %f' % corpus_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25)))

In [ ]:
#From scratch: https://stackoverflow.com/questions/56968434/bleu-score-in-python-from-scratch
import numpy as np
from collections import Counter
import math

In [ ]:
def n_gram_generator(sentence,n= 3,n_gram= False):
    '''
    N-Gram generator with parameters sentence
    n is for number of n_grams
    The n_gram parameter removes repeating n_grams
    '''
    sentence = sentence.lower()  # converting to lower case
    sent_arr = np.array(sentence.split())  # split to string arrays
    length = len(sent_arr)

    word_list = []
    for i in range(length+1):
        if i < n:
            continue
        word_range = list(range(i-n,i))
        s_list = sent_arr[word_range]
        string = ' '.join(s_list)  # converting list to strings
        word_list.append(string) # append to word_list
        if n_gram:
            word_list = list(set(word_list))
    return word_list

In [ ]:
def bleu_score(original, machine_translated):
    '''
    Bleu score function given a orginal and a machine translated sentences
    '''
    mt_length = len(machine_translated.split())
    o_length  = len(original.split())

    # Brevity Penalty
    if mt_length > o_length:
        BP=1
    else:
        penality=1-(mt_length/o_length)
        BP = np.exp(penality)

    # Clipped precision
    clipped_precision_score = []
    for ngram_level in range(1, 4):  # 1-gram to 4-gram
        
        
        original_ngram_list = n_gram_generator(original, ngram_level)
        original_n_gram = Counter(original_ngram_list)
        
        machine_ngram_list = n_gram_generator(machine_translated, ngram_level)
        machine_n_gram = Counter(machine_ngram_list)
        
        
        num_ngrams_in_translation = sum(machine_n_gram.values())  # number of ngrams in translation
        
        # iterate the unique ngrams in translation (candidate)
        for j in machine_n_gram:
            
            if j in original_n_gram:  # if found in reference
                
                if machine_n_gram[j] > original_n_gram[j]:  # CLIPPING - if found in translation more than in source, clip
                    machine_n_gram[j] = original_n_gram[j]
                    
            else:
                machine_n_gram[j] = 0

        #print (sum(machine_n_gram.values()), c)
        clipped_precision_score.append(float(sum(machine_n_gram.values())) / num_ngrams_in_translation)

    #print (clipped_precision_score)

    weights = [0.25]*4

    s = (w_i * math.log(p_i) for w_i, p_i in zip(weights, clipped_precision_score))
    s = BP * math.exp(math.fsum(s))
    return s

In [ ]:
original = train_source_list[1]
machine_translated = train_target_list[1]

print (bleu_score(original, machine_translated))

0.36787632499277756


In [ ]:
def bleu_score_2(original, machine_translated):
    '''
    Bleu score function given a orginal and a machine translated sentences
    '''
    mt_length = len(machine_translated.split())
    o_length  = len(original)

    # Brevity Penalty
    if mt_length > o_length:
        BP=1
    else:
        penality=1-(mt_length/o_length)
        BP = np.exp(penality)

    # Clipped precision
    clipped_precision_score = []
    for ngram_level in range(1, 4):  # 1-gram to 4-gram
        
        
        original_ngram_list = n_gram_generator(original, ngram_level)
        original_n_gram = Counter(original_ngram_list)
        
        machine_ngram_list = n_gram_generator(machine_translated, ngram_level)
        machine_n_gram = Counter(machine_ngram_list)
        
        
        num_ngrams_in_translation = sum(machine_n_gram.values())  # number of ngrams in translation
        
        # iterate the unique ngrams in translation (candidate)
        for j in machine_n_gram:
            
            if j in original_n_gram:  # if found in reference
                
                if machine_n_gram[j] > original_n_gram[j]:  # CLIPPING - if found in translation more than in source, clip
                    machine_n_gram[j] = original_n_gram[j]
                    
            else:
                machine_n_gram[j] = 0

        #print (sum(machine_n_gram.values()), c)
        clipped_precision_score.append(float(sum(machine_n_gram.values())) / num_ngrams_in_translation)

    #print (clipped_precision_score)

    weights = [0.25]*4

    s = (w_i * math.log(p_i) for w_i, p_i in zip(weights, clipped_precision_score))
    s = BP * math.exp(math.fsum(s))
    return s